# General Inverse Indexing

## Imports & Configs

In [1]:
import glob
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
import psycopg2
from psycopg2 import sql

In [2]:
files = [
    'test-00000-of-00001',
    'train-00000-of-00003',
    'train-00001-of-00003',
    'train-00002-of-00003',
    'validation-00000-of-00001',
]

headers = ['article', 'highlights']

batch_size = 10_000
output_file = 'combined_output.csv'

database = 'News'
table_name = 'Article'

## Parquet to CSV conversion

In [8]:
def load_file(file:str) -> None:
    parquet_file = pq.ParquetFile(f'./data/{file}.parquet')
    
    with open(f'./data/{file}.csv', 'w') as output:
        output.write(',' .join(headers) + '\n')
        
    for batch in parquet_file.iter_batches(batch_size=batch_size, columns=headers):
        df = batch.to_pandas()
        df.to_csv(f'./data/{file}.csv', mode='a', index=False, header=False)

In [9]:
for file in tqdm(files):
    loaded_rows = load_file(file)

100%|████████████████████████████████████████████| 5/5 [00:23<00:00,  4.69s/it]


## Loading data into postgres instance

In [3]:
conn = psycopg2.connect(
    dbname="News",  # not the new DB name yet
    user="admin",
    password="1234",
    host="localhost",
    port=5432
)
conn.autocommit = True  # Required for CREATE DATABASE
cur = conn.cursor()

In [4]:
cur.execute(f"""
    CREATE EXTENSION IF NOT EXISTS "uuid-ossp";

    CREATE TABLE IF NOT EXISTS {table_name} (
        id UUID PRIMARY KEY,
        article TEXT NOT NULL,
        highlights TEXT NOT NULL,
        article_vc tsvector
    );
""")

In [11]:
query = """
begin;

CREATE TEMP TABLE IF NOT EXISTS Batch (
    article TEXT NOT NULL,
    highlights TEXT NOT NULL
);

COPY Batch (article, highlights)
FROM '/sample/{file}.csv'
WITH (
    FORMAT 'csv',
    HEADER,
    DELIMITER ','
);

SELECT LEFT(article, 100), LEFT(highlights, 100)
FROM Batch
LIMIT 5;

INSERT INTO {table_name}
SELECT uuid_generate_v4(), article, highlights, to_tsvector(article || ' ' || highlights)
FROM Batch;

commit;
"""

In [13]:
for file in tqdm(files):
    cur.execute(query.format(file = file, table_name = table_name))

100%|███████████████████████████████████████████| 5/5 [22:18<00:00, 267.63s/it]
